In [ ]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)
root_dir = "/content/drive/MyDrive/"
base_dir = root_dir + "tesi_magistrale/datasets/to_zip/"
bbn_path = base_dir + "BBN/BBN/"

Mounted at /content/drive/


In [ ]:
import json

with open(bbn_path + "train.json", 'r') as inp:
    examples_train = [json.loads(l) for l in inp.readlines()]

with open(bbn_path + "test.json", 'r') as inp:
    examples_test = [json.loads(l) for l in inp.readlines()]

In [ ]:
print("il dataset train contiene ",len(examples_train), "elementi")

il dataset train contiene  32739 elementi


In [ ]:
print("il dataset test contiene ",len(examples_test), "elementi")

il dataset test contiene  6338 elementi


Merge the two dataset train e test

In [ ]:
entire_dataset_list = examples_train + examples_test
print(" uniti train e test contengono ", len(entire_dataset_list), "elementi")

 uniti train e test contengono  39077 elementi


In [ ]:
print((val*4)+1)

9769


In [ ]:
import itertools

start = 0
end = len(entire_dataset_list)//4

splitted_dataset = entire_dataset_list[start,end]
splitted_dataset = entire_dataset_list[0,val+1]

del entire_dataset_list[start, start+end]

start += end

In [ ]:
for index in range(0,):
  phrase = []
  phrases = []
  permutations = []
  num_total_phrases = 0
  for i in range(0, len(splitted_dataset)):
    all_list = []
    if len(entire_dataset_list[i]['mentions'])>1:

      # creo tutte le permutazioni
      for j in entire_dataset_list[i]['mentions']:
        all_list.append(j['labels'])
      permutations = list(itertools.product(*all_list))

      num_total_phrases += len(permutations)

      # creo tutte le possibili frasi a partire dalle permutazioni
      for p in permutations:
        phrase = entire_dataset_list[i]['tokens'].copy()
        for h in range(0, len(p)):
          phrase[entire_dataset_list[i]['mentions'][h]['start']] = p[h]
      
        phrases.append(phrase)
    
    else:
      for k in entire_dataset_list[i]['mentions'][0]['labels']:
        phrase = entire_dataset_list[i]['tokens'].copy()
        phrase[entire_dataset_list[i]['mentions'][0]['start']] = k
        phrases.append(phrase)
        num_total_phrases+=1

  del splitted_dataset
  print("numero totale di frasi", num_total_phrases)
  print("controllo se il numero di frasi totali è corretto", num_total_phrases == len(phrases))

numero totale di frasi 268420
controllo se il numero di frasi totali è corretto True


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
with open(bbn_path + "corpus_data.txt", 'w') as f:
    for s in phrases:
        f.write(str(s) + '\n')

In [ ]:
with open(bbn_path + "file.txt", 'r') as f:
    score = [line.rstrip('\n') for line in f]

print(score)